In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.{col}

In [3]:
val spark = SparkSession.builder.appName("Spark ML").getOrCreate()

spark = org.apache.spark.sql.SparkSession@34d2ec30


org.apache.spark.sql.SparkSession@34d2ec30

In [4]:
// val train_path = "file:////share/slaba05data/lab05_train.csv"
// val test_path = "file:////share/slaba05data/lab05_test.csv"

train_path = file:////share/slaba05data/lab05_train.csv
test_path = file:////share/slaba05data/lab05_test.csv


file:////share/slaba05data/lab05_test.csv

# Data Preparation

In [5]:
// var trainDf = spark
//     .read
//     .format("csv")
//     .option("header", true)
//     .option("delimeter",",")
//     .option("inferSchema", true)
//     .load(train_path)
//     //.drop("_c0", "ID")

// trainDf.show(1, 0, true)

Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3, spark-de-node-5.newprolab.com, executor 2): java.io.FileNotFoundException: File file:/share/slaba05data/lab05_train.csv does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:127)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:177)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:101)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Sou

In [7]:
var trainDf = spark
    .read
    .option("header", true)
    .option("inferSchema", true)
    .csv("/user/zarina.kamytbaeva/lab05_train.csv")
    .drop("_c0", "ID")

trainDf.show(1, 0, true)

-RECORD 0--------------------------------------------
 CR_PROD_CNT_IL              | 0                     
 AMOUNT_RUB_CLO_PRC          | 0.0                   
 PRC_ACCEPTS_A_EMAIL_LINK    | 0.0                   
 APP_REGISTR_RGN_CODE        | null                  
 PRC_ACCEPTS_A_POS           | 0.0                   
 PRC_ACCEPTS_A_TK            | 0.0                   
 TURNOVER_DYNAMIC_IL_1M      | 0.0                   
 CNT_TRAN_AUT_TENDENCY1M     | null                  
 SUM_TRAN_AUT_TENDENCY1M     | null                  
 AMOUNT_RUB_SUP_PRC          | 0.0                   
 PRC_ACCEPTS_A_AMOBILE       | 0.0                   
 SUM_TRAN_AUT_TENDENCY3M     | null                  
 CLNT_TRUST_RELATION         | null                  
 PRC_ACCEPTS_TK              | 0.0                   
 PRC_ACCEPTS_A_MTP           | 0.0                   
 REST_DYNAMIC_FDEP_1M        | 0.0                   
 CNT_TRAN_AUT_TENDENCY3M     | null                  
 CNT_ACCEPTS_TK             

trainDf = [CR_PROD_CNT_IL: int, AMOUNT_RUB_CLO_PRC: double ... 113 more fields]


lastException: Throwable = null


[CR_PROD_CNT_IL: int, AMOUNT_RUB_CLO_PRC: double ... 113 more fields]

In [8]:
val categoriesCols = List(
    "CLNT_TRUST_RELATION", 
    "APP_MARITAL_STATUS",
    "APP_KIND_OF_PROP_HABITATION", 
    "CLNT_JOB_POSITION_TYPE",
    "CLNT_JOB_POSITION", 
    "APP_DRIVING_LICENSE", 
    "APP_EDUCATION",
    "APP_TRAVEL_PASS", 
    "APP_CAR", 
    "APP_POSITION_TYPE", 
    "APP_EMP_TYPE",
    "APP_COMP_TYPE", 
    "PACK"
)

categoriesCols = List(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, CLNT_JOB_POSITION, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK)


List(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, CLNT_JOB_POSITION, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK)

In [9]:
trainDf = trainDf.drop(categoriesCols:_*).na.fill(0)

trainDf = [CR_PROD_CNT_IL: int, AMOUNT_RUB_CLO_PRC: double ... 100 more fields]


[CR_PROD_CNT_IL: int, AMOUNT_RUB_CLO_PRC: double ... 100 more fields]

In [10]:
val Array(trainDfTrain, trainDfTest) = trainDf.randomSplit(Array(0.8, 0.2))

trainDfTrain = [CR_PROD_CNT_IL: int, AMOUNT_RUB_CLO_PRC: double ... 100 more fields]
trainDfTest = [CR_PROD_CNT_IL: int, AMOUNT_RUB_CLO_PRC: double ... 100 more fields]


[CR_PROD_CNT_IL: int, AMOUNT_RUB_CLO_PRC: double ... 100 more fields]

# Train

In [11]:
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier, GBTClassifier}
import org.apache.spark.ml.feature.{VectorAssembler}

In [12]:
val vectorAssembler = new VectorAssembler()
    .setInputCols(trainDfTrain.drop("TARGET").columns)
    .setOutputCol("features")

vectorAssembler = vecAssembler_782b8c1f4fc9


vecAssembler_782b8c1f4fc9

In [13]:
val trainDfCleaned = vectorAssembler.transform(trainDfTrain).cache

trainDfCleaned = [CR_PROD_CNT_IL: int, AMOUNT_RUB_CLO_PRC: double ... 101 more fields]


[CR_PROD_CNT_IL: int, AMOUNT_RUB_CLO_PRC: double ... 101 more fields]

In [14]:
trainDfCleaned.show(1, 0, true)

-RECORD 0----------------------------------------------------------------------------------------------------
 CR_PROD_CNT_IL             | 0                                                                              
 AMOUNT_RUB_CLO_PRC         | 0.0                                                                            
 PRC_ACCEPTS_A_EMAIL_LINK   | 0.0                                                                            
 APP_REGISTR_RGN_CODE       | 0.0                                                                            
 PRC_ACCEPTS_A_POS          | 0.0                                                                            
 PRC_ACCEPTS_A_TK           | 0.0                                                                            
 TURNOVER_DYNAMIC_IL_1M     | 0.0                                                                            
 CNT_TRAN_AUT_TENDENCY1M    | 0.0                                                                            
 SUM_TRAN_

In [33]:
// Train RandomForest
// val randomForestModel = new RandomForestClassifier()
//     .setLabelCol("TARGET")
//     .setFeaturesCol("features")
//     .setSeed(42)
//     .setNumTrees(30)
//     .fit(trainDfCleaned)

randomForestModel = RandomForestClassificationModel (uid=rfc_997200060609) with 30 trees


RandomForestClassificationModel (uid=rfc_997200060609) with 30 trees

In [34]:
// val predictions = randomForestModel.transform(
//     vectorAssembler.transform(trainDfTest)
// ).cache

// predictions
//     .select(
//         "TARGET",
//         "features",
//         "rawPrediction",
//         "probability",
//         "prediction"
//     )
//     .show(5)

+------+--------------------+--------------------+--------------------+----------+
|TARGET|            features|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+--------------------+----------+
|     0|(101,[36,51,58,71...|[27.1475073068187...|[0.90491691022729...|       0.0|
|     0|(101,[36,51,58,71...|[27.1762726955721...|[0.90587575651907...|       0.0|
|     0|(101,[36,51,58,71...|[27.1762726955721...|[0.90587575651907...|       0.0|
|     0|(101,[36,51,58,71...|[27.1475073068187...|[0.90491691022729...|       0.0|
|     0|(101,[36,51,58,71...|[27.1475073068187...|[0.90491691022729...|       0.0|
+------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



predictions = [CR_PROD_CNT_IL: int, AMOUNT_RUB_CLO_PRC: double ... 104 more fields]


[CR_PROD_CNT_IL: int, AMOUNT_RUB_CLO_PRC: double ... 104 more fields]

In [40]:
val gbt = new GBTClassifier()
    .setLabelCol("TARGET")
    .setFeaturesCol("features")
    .setMaxIter(10)
    .setFeatureSubsetStrategy("auto")
    .fit(trainDfCleaned)

val predictions = gbt.transform(
    vectorAssembler.transform(trainDfTest)
    ).cache

predictions.select("TARGET","features","rawPrediction","probability","prediction").show(5)

+------+--------------------+--------------------+--------------------+----------+
|TARGET|            features|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+--------------------+----------+
|     0|(101,[36,51,58,71...|[1.23015262750714...|[0.92131179555647...|       0.0|
|     0|(101,[36,51,58,71...|[1.23015262750714...|[0.92131179555647...|       0.0|
|     0|(101,[36,51,58,71...|[1.23015262750714...|[0.92131179555647...|       0.0|
|     0|(101,[36,51,58,71...|[1.23015262750714...|[0.92131179555647...|       0.0|
|     0|(101,[36,51,58,71...|[1.23015262750714...|[0.92131179555647...|       0.0|
+------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



gbt = GBTClassificationModel (uid=gbtc_15163583b606) with 10 trees
predictions = [CR_PROD_CNT_IL: int, AMOUNT_RUB_CLO_PRC: double ... 104 more fields]


[CR_PROD_CNT_IL: int, AMOUNT_RUB_CLO_PRC: double ... 104 more fields]

In [41]:
predictions
    .select(
        "probability"
    )
    .show(1, 0, true)

-RECORD 0-----------------------------------------------
 probability | [0.9213117955564707,0.07868820444352931] 
only showing top 1 row



In [42]:
predictions.select("probability").printSchema()

root
 |-- probability: vector (nullable = true)



In [43]:
import org.apache.spark.ml.linalg.{SparseVector, Vector}
import org.apache.spark.mllib.linalg.{Vector => OldVector}

val vectorToArrayUdf = udf { vec: Any =>
    vec match {
      case v: Vector => v.toArray
      case v: OldVector => v.toArray
      case v => throw new IllegalArgumentException(
        "function vector_to_array requires a non-null input argument and input type must be " +
        "`org.apache.spark.ml.linalg.Vector` or `org.apache.spark.mllib.linalg.Vector`, " +
        s"but got ${ if (v == null) "null" else v.getClass.getName }.")
    }
  }.asNonNullable()

vectorToArrayUdf = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)


import org.apache.spark.mllib.linalg.{Vector=>OldVector}


UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)

In [44]:
predictions
    .select(vectorToArrayUdf(col("probability")).getItem(1).alias("prediction_probability"))
    .show(5)

+----------------------+
|prediction_probability|
+----------------------+
|   0.07868820444352931|
|   0.07868820444352931|
|   0.07868820444352931|
|   0.07868820444352931|
|   0.07868820444352931|
+----------------------+
only showing top 5 rows



In [21]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

In [45]:
val evaluator = new BinaryClassificationEvaluator()
    .setLabelCol("TARGET")
    .setRawPredictionCol("prediction")
    .setMetricName("areaUnderROC")

val accuracy = evaluator
    .evaluate(
        predictions.withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    )


evaluator = binEval_1b94ece8ab3d
accuracy = 0.8141813336716156


0.8141813336716156

# Predict

In [27]:
// var testDfData = spark
//     .read
//     .format("csv")
//     .option("header", true)
//     .option("delimeter",",")
//     .option("inferSchema", true)
//     .load(test_path)
// //     .drop("_c0")
//     .na.fill(0)
//     .cache
val testDfData = spark
    .read
    .option("header", true)
    .option("inferSchema", true)
    .csv("/user/zarina.kamytbaeva/lab05_test.csv")
    .na.fill(0)

testDfData.show(1, 0, true)

-RECORD 0-----------------------------------------
 _c0                         | 372289             
 ID                          | 519130             
 CR_PROD_CNT_IL              | 0                  
 AMOUNT_RUB_CLO_PRC          | 0.0                
 PRC_ACCEPTS_A_EMAIL_LINK    | 0.0                
 APP_REGISTR_RGN_CODE        | 0.0                
 PRC_ACCEPTS_A_POS           | 0.0                
 PRC_ACCEPTS_A_TK            | 0.0                
 TURNOVER_DYNAMIC_IL_1M      | 0.0                
 CNT_TRAN_AUT_TENDENCY1M     | 0.0                
 SUM_TRAN_AUT_TENDENCY1M     | 0.0                
 AMOUNT_RUB_SUP_PRC          | 0.0                
 PRC_ACCEPTS_A_AMOBILE       | 0.0                
 SUM_TRAN_AUT_TENDENCY3M     | 0.0                
 CLNT_TRUST_RELATION         | null               
 PRC_ACCEPTS_TK              | 0.0                
 PRC_ACCEPTS_A_MTP           | 0.0                
 REST_DYNAMIC_FDEP_1M        | 0.0                
 CNT_TRAN_AUT_TENDENCY3M     | 

testDfData = [_c0: int, ID: int ... 114 more fields]


lastException: Throwable = null


[_c0: int, ID: int ... 114 more fields]

In [46]:
// val realPredictions = randomForestModel.transform(vectorAssembler.transform(testDfData)).cache
val realPredictions = gbt.transform(vectorAssembler.transform(testDfData)).cache

realPredictions = [_c0: int, ID: int ... 118 more fields]


[_c0: int, ID: int ... 118 more fields]

In [47]:
realPredictions.show(1, 0, true)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 _c0                         | 372289                                                                                                                                                           
 ID                          | 519130                                                                                                                                                           
 CR_PROD_CNT_IL              | 0                                                                                                                                                                
 AMOUNT_RUB_CLO_PRC          | 0.0                                                                                                                                                              
 PRC_ACCEPTS_A_EMAIL_LINK    | 0.0 

In [48]:
realPredictions
    .withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    .select("ID", "prediction")
    .show(5)

+------+-------------------+
|    ID|         prediction|
+------+-------------------+
|519130| 0.1629910543228652|
|234045|0.07132903256005285|
|401256|0.07052035564108927|
|551070|0.09444326819568993|
|367285|0.07149831634645698|
+------+-------------------+
only showing top 5 rows



In [49]:
realPredictions
    .withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    .select("ID", "prediction")
    .withColumnRenamed("ID", "id")
    .withColumnRenamed("prediction", "target")
    .coalesce(1)
    .write
    .option("header", "true")
    .mode("overwrite")
    .format("csv")
    .option("sep" ,"\t")
    .save("lab05.csv")

In [1]:
spark.stop()

Waiting for a Spark session to start...